# Dataframe

In [1]:
import pandas as pd
import numpy as np
import Funct_modelling
df = pd.read_csv('../../preprocessing/Training/data/clean/pos_tagging/gabungan.csv')
df = df.dropna()

/var/folders/4f/xq7119ld627_2wyd6dkmwl0m0000gn/T/ipykernel_29908/4284059188.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# splitting & under sampling & vectorizing

In [2]:
x_train, x_test, y_train, y_test, vectorizer = Funct_modelling.preprocessing(df,tag=True,vector=True)

In [3]:
a = x_train.columns
a.to_series()

ab                ab
abad            abad
abadi          abadi
abah            abah
abai            abai
              ...   
zulfan        zulfan
zulfanz      zulfanz
zulhas        zulhas
zulkifli    zulkifli
zumba          zumba
Length: 15423, dtype: object

# Grid search

In [4]:
# from sklearn.naive_bayes import MultinomialNB
# nb_classifier = MultinomialNB()


In [5]:
# parameter = {
#     "alpha" : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000],
#     "fit_prior" : [True, False]
# }

# from sklearn.model_selection import GridSearchCV
# GS = GridSearchCV(estimator = nb_classifier,
#                     param_grid = parameter,
#                     scoring = "precision_micro", 
#                     verbose = 3)

# GS.fit(x_train, y_train)

In [6]:
# print(GS.best_params_)

# Modeling

In [7]:
from sklearn.naive_bayes import MultinomialNB
nb_classifier = MultinomialNB(alpha=10, fit_prior=False)
nb_classifier.fit(x_train, y_train)
y_pred = nb_classifier.predict(x_test)

# Score Naive Bayes with tag

In [8]:
print(Funct_modelling.all_report(y_test, y_pred))

              precision    recall  f1-score   support

    Negative       0.62      0.41      0.50      1782
    Positive       0.55      0.74      0.63      1720

    accuracy                           0.57      3502
   macro avg       0.58      0.58      0.56      3502
weighted avg       0.58      0.57      0.56      3502



In [9]:
import pickle

# save the model to disk
filename = 'pickle/NaiveBayes_Tag.pkl'
pickle.dump(nb_classifier, open(filename, 'wb'))

In [10]:
df_anis = pd.read_csv('../../implement/data/tag/anis.csv')
df_anis.drop('Unnamed: 0',axis=1,inplace=True)

df_prabowo = pd.read_csv('../../implement/data/tag/prabowo.csv')
df_prabowo.drop('Unnamed: 0',axis=1,inplace=True)

df_ganjar = pd.read_csv('../../implement/data/tag/ganjar.csv')
df_ganjar.drop('Unnamed: 0',axis=1,inplace=True)

In [11]:
dic = [0,1,2,3,4,5,6,7,8,9]
def clean_number(data):
    data = str(data)
    data = data.split()
    temp = []
    for i in data:
        
        num = False
        for j in dic:
            if str(j) in i:
                num = True
                break
        if not num:
            temp.append(i)

    return ' '.join(temp)

In [12]:
def vector(data,vectorizer):
    data = vectorizer.transform(data)
    data = data.toarray()
    data = pd.DataFrame(data, columns=vectorizer.get_feature_names_out())
    return data

In [13]:
tag_anis = df_anis[df_anis.columns[-20:]]
X_anis = df_anis['no_stopwords'].apply(clean_number)
X_anis = vector(X_anis,vectorizer=vectorizer)

tag_prabowo = df_prabowo[df_prabowo.columns[-20:]]
X_prabowo = df_prabowo['no_stopwords'].apply(clean_number)
X_prabowo = vector(X_prabowo,vectorizer=vectorizer)

tag_ganjar = df_ganjar[df_ganjar.columns[-20:]]
X_ganjar = df_ganjar['no_stopwords'].apply(clean_number)
X_ganjar = vector(X_ganjar,vectorizer=vectorizer)

In [14]:
X_anis = pd.concat([X_anis,tag_anis],axis=1)
X_prabowo = pd.concat([X_prabowo,tag_prabowo],axis=1)
X_ganjar = pd.concat([X_ganjar,tag_ganjar],axis=1)

In [15]:
df_anis['Sentiment'] = nb_classifier.predict(X_anis)
df_prabowo['Sentiment'] = nb_classifier.predict(X_prabowo)
df_ganjar['Sentiment'] = nb_classifier.predict(X_ganjar)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- cc
- cd
- fw
- in
- jj
- ...
